In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import torchvision
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 12, 5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(12)
        self.conv5 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(24)
        self.conv7 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*34*34, 4096)
        self.fc2 = nn.Linear(28000, 4096)
        self.fc3 = nn.Linear(4096, 14)
        self.fc4 = nn.Linear(1024, 14)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))      
        output = F.relu(self.bn2(self.conv2(output)))     
        output = self.pool(output)                        
        output = F.relu(self.bn4(self.conv4(output)))     
        output = F.relu(self.bn5(self.conv5(output)))
        output = self.pool(output)
        output = F.relu(self.bn6(self.conv6(output)))
        output = F.relu(self.bn7(self.conv7(output)))
        output = self.pool(output)
        output = output.view(-1, 24*34*34)
        output = F.relu(self.fc1(output))
        # output = F.relu(self.fc2(output))
        # output = F.relu(self.fc3(output))
        output = self.fc3(output) 

        return output

In [3]:
def saveModel(model, accuracy, path='./model_large_{}.pth'):
    torch.save(model.state_dict(), path.format(accuracy))

In [4]:
def testAccuracy(model, outputs, labels):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    correct = 0.0
    
    _, predicted = torch.max(outputs.data, 1)
    with torch.no_grad():
        for p, l in zip(predicted, labels):
            total += 1
            if p.data == l.data:
                correct += 1
    
    accuracy = 100 * (correct / total)
    return accuracy

In [5]:
def train(device, limit=5000, batch_count=20, epoch=200, learning_rate=0.005, save_path='./ososa_model.pth'):
    model = Network().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    batch_size = int(limit / batch_count)
    best_accuracy = 0

    for e in range(epoch):
        total_accuracy = 0
        for b in range(batch_count):
            model.train()
            start_index = b*batch_size
            end_index = (b+1)*batch_size

            batched_x, batched_y = take_data(start_index, end_index)

            train_data = torch.from_numpy(batched_x).float().to(device)
            train_labels = torch.from_numpy(batched_y).long().to(device)
            optimizer.zero_grad()
            prediction = model(train_data)
            loss = loss_fn(prediction, train_labels)
            loss.backward()
            optimizer.step()
                
            accuracy = testAccuracy(model, prediction, train_labels)
            total_accuracy += accuracy

            batch_limit = 20
            if b % batch_limit == batch_limit - 1:
                current_accuracy = total_accuracy / batch_limit
                total_accuracy = 0

                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    print(best_accuracy)
                    saveModel(model, accuracy)
        if e % 10 == 0:
            print("Loss in e={} is {}".format(e, loss))

    print('finished')

In [6]:
def test(start_limit, end_limit, device, batch_count=4, load_path='./model_large.pth'):
    correct = 0
    total = 0
    
    batch_size = int((end_limit - start_limit) / batch_count)
    for b in range(batch_count):

        start_index = b*batch_size + start_limit
        end_index = (b+1)*batch_size + start_limit
        
        batched_x, batched_y = take_data(start_index, end_index)

        test_data = torch.from_numpy(batched_x).float().to(device)
        test_labels = torch.from_numpy(batched_y).long().to(device)
        with torch.no_grad():
            model = Network().to(device)
            model.load_state_dict(torch.load(load_path))
            outputs = model(test_data)
            _, predicted = torch.max(outputs.data, 1)
            for p, l in zip(predicted, test_labels):
                total += 1
                if p.data == l.data:
                    correct += 1
        if b % 10 == 9:
            print("Accuracy: {}\tTotal: {}\tCorrect: {}".format(correct/total, total, correct))
    print("Total:", total)
    print("Correct:", correct)
    print("Accuracy:", correct/total)
    torch.cuda.empty_cache()

In [7]:
def take_data(start_index, end_index, input_directory_1="./numpy_images", input_directory_2="./labels"):
	directory_list = os.listdir(input_directory_1)[start_index: end_index]
	data = []
	labels = []
	for i, image_name in enumerate(directory_list):
		try:
			datum = np.load("{}/{}".format(input_directory_1, image_name))
			datum = datum.reshape(3, 300, 300)
			label = np.load("{}/{}".format(input_directory_2, image_name))
			data.append(datum)
			labels.append(label)
		except ValueError:
			pass
	return np.array(data), np.array(labels)

In [8]:
if torch.cuda.is_available:
    device = 'cuda'
else:
    device = 'cpu'

In [9]:

train(device, limit=60000, batch_count=2000, epoch=200)
torch.cuda.empty_cache()

11.686371100164203
16.54597701149425
22.747126436781606
24.109195402298845
26.83169129720854
26.856321839080465
26.96551724137931
28.154761904761905
29.25656814449918
30.902298850574716
32.9712643678161
33.00574712643677
33.05172413793103
34.2816091954023
35.222085385878486
35.39080459770115
37.47126436781609
Loss in e=0 is 1.8447433710098267
38.373563218390814
38.43678160919539
38.5919540229885
39.439655172413794
40.206896551724135
41.781609195402304
42.46551724137931
44.241379310344826
45.34482758620688
46.488505747126425
47.32183908045977
48.2816091954023
49.310344827586206
50.465927750410515
51.132183908045974
51.431034482758626
52.28160919540229
52.52380952380953
53.252873563218394
55.00574712643678
56.045977011494244
56.919540229885044
57.74302134646962
57.90229885057471
58.67241379310345
59.98275862068965
61.718390804597696
62.172413793103445
65.1912972085386
Loss in e=10 is 1.528051733970642
66.73481116584564
69.55336617405582
72.16666666666669
74.49425287356323
75.994252873563

KeyboardInterrupt: 

In [12]:
test(60000, 61000, device, batch_count=10, load_path='./model_large_46.66.pth')
torch.cuda.empty_cache()

Accuracy: 0.38329979879275655	Total: 994	Correct: 381
Total: 994
Correct: 381
Accuracy: 0.38329979879275655


In [11]:

torch.cuda.empty_cache()